<a href="https://colab.research.google.com/github/hochthom/OG-AI4Artists-2022/blob/main/Image_Generation/StyleGAN2_ADA_Image_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Images with StyleGAN2 on Colab



First we check for what GPU we have been assigned.

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-cd474c45-2988-8a80-bf84-1e298348f391)


Next we connect to Google drive account:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!ls /content/drive/

MyDrive


In [ ]:
!pip freeze

In [3]:
!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

     |████████████████████████████████| 108 kB 28.5 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
     |████████████████████████████████| 12.8 MB 53.1 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.


# Install StyleGAN2 Repo

In [4]:
HOME_DIR = '/content/drive/MyDrive/AI4Artists/OG-AI4Artists-2022'
!ls $HOME_DIR

'AI for Artists - Img-Gen.gslides'    notebooks
'AI for Artists - Intro.gslides'      stylegan2-ada-pytorch
'AI for Artists - Text-Gen.gslides'


In [5]:
import os

sg2_dir = os.path.join(HOME_DIR, 'stylegan2-ada-pytorch')
if os.path.isdir(sg2_dir):
    %cd $sg2_dir
elif os.path.isdir(HOME_DIR):
    #install script
    %cd $HOME_DIR
    !git clone https://github.com/hochthom/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !mkdir samples
else:
  print('Set valid directory for HOME_DIR!')

/content/drive/MyDrive/AI4Artists/OG-AI4Artists-2022/stylegan2-ada-pytorch


In [ ]:
!ls

# Model Preparation
Upload your model (*.pkl file) into the pretrained folder in your local repo (the one we just generated) or train a model from scratch (see other notebook).

You can use also a pretrained model from this awesome list: 
https://github.com/justinpinkney/awesome-pretrained-stylegan2

# Image Creation
First we have to set some variables ...

* `dataset_path`: this is the path to your .zip file
* `resume_from`: if you’re starting a new dataset I recommend `'ffhq256'` or `'./pretrained/wikiart.pkl'`


In [6]:
#required: definitely edit these!
#model = './pretrained/StyleGAN2_metFaces_res1024.pkl'  # humane faces from art works
model = './pretrained/StyleGAN2_ffhq_res512.pkl'        # flickr faces high quality
outdir = './samples'

if not os.path.isdir(outdir):
  print('Warning: Could not find directory for saving generated images!')

To see all available parameters for training you can execute the following code

In [ ]:
!python generate.py --help

Finally the code for generting images :)

In [8]:
!python generate.py --outdir=$outdir --trunc=0.8 --seeds=0-5 --network=$model

Loading networks from "./pretrained/StyleGAN2_ffhq_res512.pkl"...
Generating image for seed 0 (0/6) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 1 (1/6) ...
Generating image for seed 2 (2/6) ...
Generating image for seed 3 (3/6) ...
Generating image for seed 4 (4/6) ...
Generating image for seed 5 (5/6) ...


# Experiments

## Truncation
What is the influence of the truncation parameter? A value between -1 to 1 will give pretty realistic images. But the higher the value we set, the weirder it gets.

In [ ]:
!python generate.py --outdir=$outdir --process="truncation" --start=-0.5 --stop=3 --increment=0.02 --seeds=5 --network=$model

## Interpolation
Starting from one image, we want to gradually move to another image ( akind of morphing effect). We can define the sequence with our seed parameter. The number of frames per interpolation can be set with the *--frames* parameter. E.g. for *--seed=1,5,1* the number of frames will be two times the *--frames* value (from 1 to 5 and back to 1)! 

In [ ]:
!python generate.py --outdir=$outdir --process="interpolation" --frames=100 --trunc=0.8 --seeds=0,5 --network=$model

Other fun way of interpolation is through a **noise / circular loop** in the latent space of the model. You can control the diversity of the generated samples with the *--diameter* parameter. Circle loop needs higher values (100) than noise loop (1) but good values also strongly depend on the used model!

In [ ]:
!python generate.py --outdir=$outdir --process="interpolation" --interpolation="circularloop" --diameter=100 --frames=240 --trunc=0.8 --random_seed=0 --network=$model

## Projection
Is the process to find a vector that generates an image that is perceptionally close to the given one.

*   `--target`: this is a path to the image file that we want to "find" with our model. This image must be the exact same size as the images used for training the model.
*   `--num-steps`: how many iterations the projector should use to find a good projection. Higher values will take longer but will likely find better fitting images.

In [ ]:
!python projector.py --help

In [15]:
target = './downloads/art-hauntington-unsplash-512.jpg'
#target = './downloads/mathias-konrath-unsplash_512.jpg'

In [16]:
!python projector.py --outdir=$outdir --target=$target --num-steps=200 --seed=0 --network=$model

Loading networks from "./pretrained/StyleGAN2_ffhq_res512.pkl"...
Computing W midpoint and stddev using 10000 samples...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
step    1/200: dist 0.60 loss 9557.81
step    2/200: dist 0.57 loss 13249.50
step    3/200: dist 0.51 loss 11521.82
step    4/200: dist 0.57 loss 8737.99
step    5/200: dist 0.45 loss 5736.14
step    6/200: dist 0.49 loss 3210.26
step    7/200: dist 0.39 loss 2003.14
step    8/200: dist 0.46 loss 2411.67
step    9/200: dist 0.48 loss 4028.77
step   10/200: dist 0.36 loss 5837.86
step   11/200: dist 0.34 loss 6856.03
step   12/200: dist 0.39 loss 6585.55
step   13/200: dist 0.38 loss 5787.26
step   14/200: dist 0.33 loss 4627.46
step   15/200: dist 0.38 loss 3462.34
step   16/200: dist 0.33 loss 2563.82
step   17/200: dist 0.34 loss 2089.59
step   18/200: dist 0.36 loss 1789.95
step   19/200: dist 0.36 loss 1552.74
step   20/200: dist 0.36 loss 1498.76
step   21/

For interpolation we first have to combine the generated projection files (*.npz).

In [20]:
!python combine_npz.py --outdir=$outdir --npzs='./samples/woman.npz,./samples/old_man.npz'

Combining .npz files...
./samples/woman.npz
./samples/old_man.npz
torch.Size([2, 16, 512])


In [21]:
!python generate.py --outdir=$outdir --process=interpolation --frames=240 --space=w --projected-w=./samples/combined.npz --network=$model

Loading networks from "./pretrained/StyleGAN2_ffhq_res512.pkl"...
Generating image for frame 0/240 ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/240 ...
Generating image for frame 2/240 ...
Generating image for frame 3/240 ...
Generating image for frame 4/240 ...
Generating image for frame 5/240 ...
Generating image for frame 6/240 ...
Generating image for frame 7/240 ...
Generating image for frame 8/240 ...
Generating image for frame 9/240 ...
Generating image for frame 10/240 ...
Generating image for frame 11/240 ...
Generating image for frame 12/240 ...
Generating image for frame 13/240 ...
Generating image for frame 14/240 ...
Generating image for frame 15/240 ...
Generating image for frame 16/240 ...
Generating image for frame 17/240 ...
Generating image for frame 18/240 ...
Generating image for frame 19/240 ...
Generating image for frame 20/240 ...
Generating image for frame 21/240 ...
G